In [90]:
import re
import pandas as pd

In [91]:

with open("bulk_results.txt",'r') as resultsfile:
    data = resultsfile.readlines()
data = [d.strip() for d in data]

In [92]:
len(data)

989

In [93]:
data[:35]

['',
 '------------------------------',
 'ProsperLoans',
 '------------------------------',
 '3394979',
 '801 nodes in node_set',
 '6259 original edges',
 'originally 4694 edges in the train set',
 'originally 1565 edges in the test set',
 '0 edges in the scc',
 '0 edges in the test set',
 '1 nodes in the graph',
 '',
 '',
 'There are 0 edges to predict',
 '',
 'Something went wrong',
 '------------------------------',
 'DutchCollege',
 '------------------------------',
 '3062',
 '32 nodes in node_set',
 '710 original edges',
 'originally 532 edges in the train set',
 'originally 178 edges in the test set',
 '515 edges in the scc',
 '127 edges in the test set',
 '29 nodes in the graph',
 '',
 '',
 'There are 127 edges to predict',
 '',
 '',
 'Normalized Effective Transition 127',
 'time 0.0865333080291748']

In [52]:
okay = re.match(predictor,data[33])

In [57]:
print(okay.groups()[0])

Normalized Effective Transition 


In [97]:
nodes = re.compile(r'^([0-9]*) nodes in the graph')
train_edges = re.compile(r'^([0-9]*) edges in the scc')
test_edges = re.compile(r'^([0-9]*) edges in the test')
predictor = re.compile(r'^([A-Z][A-Za-z\s]*)[0-9]*$')
score = re.compile(r'^score ([0-9\.]{3,7})[0-9]*$')

In [99]:
results = {}
line = 0
while line < len(data):
    if data[line].startswith('----'):
        name = data[line+1]
        results[name] = {}
        line += 3
        while not data[line+1].startswith('----'):
            check_nodes = re.match(nodes,data[line])
            check_train = re.match(train_edges,data[line])
            check_test = re.match(test_edges,data[line])
            meth_test = re.match(predictor,data[line])
            
            
            if check_nodes:
                results[name]["Nodes"] = check_nodes.groups()[0]
            elif check_train:
                results[name]["Train Edges"] = check_train.groups()[0]
            elif check_test:
                results[name]["Test Edges"] = check_test.groups()[0]
            elif meth_test:
                line += 2
                score_test = re.match(score,data[line])
                tup = (meth_test.groups()[0], score_test.groups()[0])
                results[name][tup[0]] = tup[1]
            
            line += 1
            if line == len(data)-1:
                break
    line += 1
        

In [100]:
df = pd.DataFrame.from_dict(results,orient='index')

In [101]:
columns = ["Train Edges","Test Edges","Node","Normalized ET","Standard ET","Shortest Path","Katz","Hitting Time","del1","del2","Normalized ETA","Standard ETA"]

In [102]:
df.columns = columns

In [107]:
df.drop(['del1','del2'],axis=1,inplace=True)
df.drop(['Digg','ProsperLoans'],inplace=True)

In [108]:
df

,Train Edges,Test Edges,Node,Normalized ET,Standard ET,Shortest Path,Katz,Hitting Time,Normalized ETA,Standard ETA
AskUbuntu,4761,1154,550,0.03292,0.04506,0.02079,0.08578,0.00346,0.02426,0.08838
Chess,7130,2348,735,0.02385,0.10477,0.01618,0.11584,0.02342,0.02810,0.11030
DNC,1784,646,220,0.08978,0.02167,0.05108,0.21517,0.00154,0.09907,0.18575
DiggFriends,11,3,5,0.66666,0.66666,0.0,0.66666,0.66666,0.66666,0.66666
DutchCollege,515,127,29,0.44881,0.61417,0.51968,0.64566,0.45669,0.47244,0.62992
Enron,5054,1512,362,0.06944,0.09060,0.01719,0.13492,0.02843,0.07870,0.12169
Epinions,2048,759,274,0.03293,0.01976,0.03952,0.10540,0.01317,0.04874,0.09617
FacebookWSON,661,134,236,0.03731,0.03731,0.02985,0.05223,0.02985,0.04477,0.05970
Linux,22720,7579,646,0.00633,0.24198,0.01147,0.27800,0.01556,0.09315,0.27892
Slashdot,599,167,222,0.0,0.02994,0.00598,0.03592,0.0,0.0,0.04191
